# Trabajo Practico 2

In [3]:
import numpy as np
import simpy

## Ejercicio 1

Simule el siguiente problema.
Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.
El tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media 𝜇
Considerar 𝜇 = 1, 2 𝑦 4 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠

Realizar 100 simulaciones de cada modelo, con 100000 solicitudes procesadas, y determinar:
- El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión se
cae por timeout).
- La fracción de las solicitudes que no esperaron para ser procesadas.
- La tasa de finalización de consultas (consultas finalizadas por segundo)
- ¿Qué solución le recomienda? Justifique

In [4]:
def tiempos_de_arribo(media, cantidad_ensayos):
    z = np.random.exponential(media, cantidad_ensayos)
    return np.concatenate(([0], np.cumsum(z)), axis=None)

#### Alternativa 1
Se utilizan 2 bases de datos distribuidas.
Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base A y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la
base de datos B.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias,
𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 0,95 𝑠𝑒𝑔 respectivamente.

In [5]:
def tiempos_en_base(arribos, media):
    t_actual = 0
    tiempos_espera = []
    tiempos_procesado = []
    for t_arribo in arribos:
        t_base = np.random.exponential(media)
        if t_arribo < t_actual:
            t_espera = t_actual - t_arribo
            tiempos_espera.append(t_espera)
            t_procesado = t_arribo + t_espera + t_base
            tiempos_procesado.append(t_procesado)
            t_actual += t_base
        else:
            t_espera = 0
            tiempos_espera.append(t_espera)
            t_procesado = t_arribo + t_base
            tiempos_procesado.append(t_procesado)
            t_actual = t_arribo + t_base
    return tiempos_espera, tiempos_procesado

In [6]:
def alternativa_1(media_arribos, solicitudes_procesadas):
    tiempos_arribo = tiempos_de_arribo(media_arribos, solicitudes_procesadas)
    proba_a = 0.6
    tiempos_a = []
    tiempos_b = []
    for t in tiempos_arribo:
        bdd = np.random.uniform()
        t_p = t
        t_e = 0
        if bdd < proba_a:
            tiempos_a.append(t)
        else:
            tiempos_b.append(t)
    tiempos_espera_a,tiempos_procesado_a = tiempos_en_base(tiempos_a, 0.7)
    tiempos_espera_b,tiempos_procesado_b = tiempos_en_base(tiempos_b, 0.95)
    return tiempos_a + tiempos_b, tiempos_procesado_a + tiempos_procesado_b, tiempos_espera_a + tiempos_espera_b

In [7]:
def simular(media_arribos, cantidad_simulaciones, muestra, alternativa):
    tiempos_medios_espera_resultado = []
    fraccion_sin_espera_resultado = []
    tasa_finalizacion_resultado = []
    for i in range(cantidad_simulaciones):
        tiempos_arribo, tiempos_procesado, tiempos_espera = alternativa(media_arribos, muestra)
        tiempos_medios_espera_resultado.append(sum(tiempos_espera)/muestra)
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        tasa_finalizacion = muestra / max(tiempos_procesado)
        tasa_finalizacion_resultado.append(tasa_finalizacion)
    tiempo_medio_total = sum(tiempos_medios_espera_resultado)/len(tiempos_medios_espera_resultado)
    fraccion_sin_espera_total = sum(fraccion_sin_espera_resultado)/len(fraccion_sin_espera_resultado)
    tasa_finalizacion_total = sum(tasa_finalizacion_resultado)/len(tasa_finalizacion_resultado)
    return tiempo_medio_total, fraccion_sin_espera_total, tasa_finalizacion_total

In [8]:
def mostrar_resultado(resultado):
    tiempo_medio_espera, fraccion_sin_espera, tasa_finalizacion = resultado
    print("Tiempo medio de espera: {} segundos".format(tiempo_medio_espera))
    print("Fraccion de solicitudes sin esperar: {}".format(fraccion_sin_espera))
    print("Tasa de finalizacion: {} solicitudes finalizadas por segundo".format(tasa_finalizacion))

In [9]:
mostrar_resultado(simular(1, 100, 100000, alternativa_1))

Tiempo medio de espera: 0.5375925702557873 segundos
Fraccion de solicitudes sin esperar: 0.5959108999999999
Tasa de finalizacion: 0.9997621163170294 solicitudes finalizadas por segundo


In [14]:
mostrar_resultado(simular(2, 100, 100000, alternativa_1))

Tiempo medio de espera: 0.20055645877933206 segundos
Fraccion de solicitudes sin esperar: 0.7982833
Tasa de finalizacion: 0.49967017988773416 solicitudes finalizadas por segundo


In [15]:
mostrar_resultado(simular(4, 100, 100000, alternativa_1))

Tiempo medio de espera: 0.0890836153119197 segundos
Fraccion de solicitudes sin esperar: 0.8990539000000003
Tasa de finalizacion: 0.24988685947578199 solicitudes finalizadas por segundo


#### Alternativa 2
Utilizar 1 base de datos central.
En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜

In [117]:
def alternativa_2(media_arribos, solicitudes_procesadas):
    tiempos_arribo = tiempos_de_arribo(media_arribos, solicitudes_procesadas)
    tiempos_espera,tiempos_procesado = tiempos_en_base(tiempos_arribo, 0.8)
    return tiempos_arribo, tiempos_procesado, tiempos_espera

In [118]:
mostrar_resultado(simular(1, 100, 100000, alternativa_2))

Tiempo medio de espera: 3.2164751593019743 segundos
Fraccion de solicitudes sin esperar: 0.19988480000000003
Tasa de finalizacion: 0.9995066340810936 solicitudes finalizadas por segundo


In [131]:
mostrar_resultado(simular(2, 100, 100000, alternativa_2))

Tiempo medio de espera: 0.5338943495503664 segundos
Fraccion de solicitudes sin esperar: 0.5998832
Tasa de finalizacion: 0.5001211553168818 solicitudes finalizadas por segundo


In [132]:
mostrar_resultado(simular(4, 100, 100000, alternativa_2))

Tiempo medio de espera: 0.19990414086428815 segundos
Fraccion de solicitudes sin esperar: 0.8001175999999998
Tasa de finalizacion: 0.2500092621084846 solicitudes finalizadas por segundo


#### Conclusiones

Para concluir cuál alternativa es la que se recomienda se deben observar los distintos valores obtenidos:
- Tiempo medio de espera: la alternativa 1 tiene menor tiempo medio de espera para todas las medias propuestas
- Fracción de solicitudes sin esperar: la alternativa 1 tiene mayor fracción de solicitudes sin esperar para todas las medias propuestas
- Tasa de finalización: son similares en ambas alternativas

En conclusión, se recomienda la alternativa 1 ya que se obtuvieron mejores resultados en los distintos valores analizados. 

## Ejercicio 4

Implementar las 2 alternativas del Ejercicio 1 utilizando SimPy.

Comparar los resultados obtenidos con el ejercicio 1.

In [21]:
# common
MUESTRA = 1000 # todo: setear bien muestra
N_SIMULACIONES = 100

def request(env, counter, media_base, tiempos_espera, tiempos_procesado):
    tiempo_arribo = env.now

    with counter.request() as req:
        yield req
        tiempo_actual = env.now
        tiempo_espera = tiempo_actual - tiempo_arribo
        t_base = np.random.exponential(media_base)
        yield env.timeout(t_base)
        tiempos_espera.append(tiempo_espera)
        tiempos_procesado.append(tiempo_actual + tiempo_espera + t_base)

def calcular_promedio(numeros):
    return sum(numeros) / len(numeros)

#### Alternativa 1
Se utilizan 2 bases de datos distribuidas.
Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base A y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la
base de datos B.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias,
𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 0,95 𝑠𝑒𝑔 respectivamente.

In [22]:
PROBA_BASE_A = 0.6
MEDIA_BASE_A = 0.7
MEDIA_BASE_B = 0.95

def source_1(env, muestra, media_arribos, proba_a, counter_a, counter_b, media_base_a, media_base_b, tiempos_espera, tiempos_procesado):
    for i in range(muestra):
        bdd = np.random.uniform()
        if bdd < proba_a:
            r = request(env, counter_a, media_base_a, tiempos_espera, tiempos_procesado)
        else:
            r = request(env, counter_b, media_base_b, tiempos_espera, tiempos_procesado)
        env.process(r)
        t = np.random.exponential(media_arribos)
        yield env.timeout(t)
        
def alternativa_1(muestra, media_arribos, proba_a, media_base_a, media_base_b, n_simulaciones):
    tiempos_espera_medios_resultado = []
    fraccion_sin_espera_resultado = []
    taza_finalizacion_resultado = []
    
    for _ in range(n_simulaciones):
        tiempos_espera = []
        tiempos_procesado = []
        env = simpy.Environment()
        counter_a = simpy.Resource(env, capacity=1)
        counter_b = simpy.Resource(env, capacity=1)
        env.process(source_1(env, muestra, media_arribos, proba_a, counter_a, counter_b, media_base_a, media_base_b, tiempos_espera, tiempos_procesado))
        env.run()        
        
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        
        taza_finalizacion = muestra / max(tiempos_procesado)
        taza_finalizacion_resultado.append(taza_finalizacion)
        
        tiempos_espera_medios_resultado.append(calcular_promedio(tiempos_espera))
    return tiempos_espera_medios_resultado, fraccion_sin_espera_resultado, taza_finalizacion_resultado

In [23]:
MEDIA_ARRIBOS = 1

tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_1(MUESTRA, MEDIA_ARRIBOS, PROBA_BASE_A, MEDIA_BASE_A, MEDIA_BASE_B, N_SIMULACIONES)
tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)

mostrar_resultado((tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion))

Tiempo medio de espera: 0.5383027685533895 segundos
Fraccion de solicitudes sin esperar: 0.5973900000000002
Tasa de finalizacion: 0.9985338208104517 solicitudes finalizadas por segundo


In [24]:
MEDIA_ARRIBOS = 2

tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_1(MUESTRA, MEDIA_ARRIBOS, PROBA_BASE_A, MEDIA_BASE_A, MEDIA_BASE_B, N_SIMULACIONES)
tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)

mostrar_resultado((tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion))

Tiempo medio de espera: 0.20843633107364312 segundos
Fraccion de solicitudes sin esperar: 0.7975900000000001
Tasa de finalizacion: 0.5020283596062306 solicitudes finalizadas por segundo


In [25]:
MEDIA_ARRIBOS = 4

tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_1(MUESTRA, MEDIA_ARRIBOS, PROBA_BASE_A, MEDIA_BASE_A, MEDIA_BASE_B, N_SIMULACIONES)
tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)

mostrar_resultado((tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion))

Tiempo medio de espera: 0.09025496746887779 segundos
Fraccion de solicitudes sin esperar: 0.8981900000000005
Tasa de finalizacion: 0.25167317684826346 solicitudes finalizadas por segundo


#### Alternativa 2

Utilizar 1 base de datos central. En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜

In [28]:
MEDIA_BASE = 0.8

def source_2(env, muestra, media_arribos, counter, media_base, tiempos_espera, tiempos_procesado):
    for i in range(muestra):
        r = request(env, counter, media_base, tiempos_espera, tiempos_procesado)
        env.process(r)
        t = np.random.exponential(media_arribos)
        yield env.timeout(t)
        
def alternativa_2(muestra, media_arribos, media_base, n_simulaciones):
    tiempos_espera_medios_resultado = []
    fraccion_sin_espera_resultado = []
    taza_finalizacion_resultado = []
    
    for _ in range(n_simulaciones):
        tiempos_espera = []
        tiempos_procesado = []
        env = simpy.Environment()
        counter = simpy.Resource(env, capacity=1)
        env.process(source_2(env, muestra, media_arribos, counter, media_base, tiempos_espera, tiempos_procesado))
        env.run()        
        
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        
        taza_finalizacion = muestra / max(tiempos_procesado)
        taza_finalizacion_resultado.append(taza_finalizacion)
        
        tiempos_espera_medios_resultado.append(calcular_promedio(tiempos_espera))
    return tiempos_espera_medios_resultado, fraccion_sin_espera_resultado, taza_finalizacion_resultado

def calcular_promedio(numeros):
    return sum(numeros) / len(numeros)

In [29]:
MEDIA_ARRIBOS = 1

tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_2(MUESTRA, MEDIA_ARRIBOS, MEDIA_BASE, N_SIMULACIONES)
tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)

mostrar_resultado((tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion))

Tiempo medio de espera: 3.1645439832990894 segundos
Fraccion de solicitudes sin esperar: 0.20122999999999988
Tasa de finalizacion: 0.9917811438138989 solicitudes finalizadas por segundo


In [30]:
MEDIA_ARRIBOS = 2

tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_2(MUESTRA, MEDIA_ARRIBOS, MEDIA_BASE, N_SIMULACIONES)
tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)

mostrar_resultado((tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion))

Tiempo medio de espera: 0.5152181126183145 segundos
Fraccion de solicitudes sin esperar: 0.60503
Tasa de finalizacion: 0.49955776137876357 solicitudes finalizadas por segundo


In [31]:
MEDIA_ARRIBOS = 4

tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_2(MUESTRA, MEDIA_ARRIBOS, MEDIA_BASE, N_SIMULACIONES)
tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)

mostrar_resultado((tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion))

Tiempo medio de espera: 0.2038985571536843 segundos
Fraccion de solicitudes sin esperar: 0.7984299999999996
Tasa de finalizacion: 0.25156314835329285 solicitudes finalizadas por segundo


#### Conclusiones

Al igual que en el ejercicio 1 la mejor alternativa es la 1.